### Importing all dependencies

In [ ]:
from pydaaltensorflow import DAALNet
import pydaaltensorflow as pydaal
import tensorflow as tf
import numpy as np
import os

### Define necessary primitives and functions to create a simplified Convolutionl Neural Net (CNN)

In [ ]:
def get_variable(name, shape):
  return tf.get_variable(name, shape, initializer=tf.random_normal_initializer(seed=0))

In [ ]:
def conv2d(x, kernel, in_ch, out_ch, strides=[1, 1, 1, 1]):
    W = get_variable(name='wc', shape=[kernel[0], kernel[1], in_ch, out_ch])
    b = get_variable(name='bc', shape=[out_ch])
    # Conv2D wrapper, with bias and relu activation
    conv2d = tf.nn.conv2d(x, W, strides=strides, padding='SAME')
    conv2d = tf.nn.bias_add(conv2d, b)
    return tf.nn.relu(conv2d)

In [ ]:
def maxpool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1]): # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='VALID')

In [ ]:
def avgpool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1]): # AvgPool2D wrapper
    return tf.nn.avg_pool(x, ksize=ksize, strides=strides, padding='SAME')

In [ ]:
def fully_connected(x, w_shape, b_shape):
    W = get_variable(name='wd', shape=w_shape)
    b = get_variable(name='bd', shape=b_shape)
    # Fully connected layer with added biases
    return tf.matmul(x, W) + b

### Define a small CNN model which would be transformed to (Py)DAAL

In [ ]:
def conv_net(x, n_classes):
    # 1st Convolution Layer
    with tf.variable_scope('conv1') as scope:
        conv1 = conv2d(x, [5, 5], 3, 32)
        # Max Pooling (down-sampling)
        conv1 = maxpool2d(conv1)

    # 2nd Convolution Layer
    with tf.variable_scope('conv2') as scope:
        conv2 = conv2d(conv1, [5, 5], 32, 64)
        # Avg Pooling (down-sampling)
        conv2 = avgpool2d(conv2)
        conv2 = tf.reshape(conv2, [-1, 7*7*64])

    # 1st Fully Connected Layer
    with tf.variable_scope('full1') as scope:
        full1 = fully_connected(conv2, [7*7*64, 1024], [1024])
    
    # Output, class prediction
    with tf.variable_scope('full2') as scope:
        full2 = fully_connected(full1, [1024, n_classes], [n_classes])

    return full2

### Define a proper TensorFlow environment and run and dump to the file the transformed model

In [ ]:
def run_case(data, checkpoint_dir):
    # reshape data to TF format
    data = np.transpose(data, (0, 2, 3, 1))

    # Network Parameters
    n_classes = 10

    # reset the Graph
    tf.reset_default_graph()

    # tf Graph input
    x = tf.placeholder(tf.float32, shape=data.shape)

    # Construct model
    pred = conv_net(x, n_classes)

    # Transform to the Intel DAAL model
    model = pydaal.transform(pred)

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Provide a reference path to PyDAAL model
    pydaal.dump_model(model, checkpoint_dir)

    # Create a saver 
    saver = tf.train.Saver(tf.trainable_variables(), max_to_keep = 0)
    
    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        checkpoint_path = os.path.join(checkpoint_dir, 'model.ckpt')
        saver.save(sess, checkpoint_path, global_step = 0)

        predictions  = sess.run(pred, feed_dict={x: data})

    return predictions

### Create some superficial data and run the above TensorFlow model

In [ ]:
test_data = np.random.randn(1, 3, 28, 28)/100
tf_predictions = run_case(test_data, '/tmp')

### Build a net from the dumped model (stored in the checkpoint directory)

In [ ]:
net = DAALNet().build('/tmp')

### Run forward pass again to obtain predictions

In [ ]:
with net.predict(test_data) as predictions:
    print('DAAL predictions: %s' % predictions)
    print('TF predictions  : %s' % tf_predictions)